<a href="https://colab.research.google.com/github/ANQI-70/stock_market_analytics/blob/main/Stock_Analytics_HM_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install yfinance

In [41]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

# HOME ASSIGNMENT PAGE https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/cohorts/2024/homework%202.md

# Question 1

In [42]:
#get IP0 2023 Data
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}
url = "https://stockanalysis.com/ipos/filings/"

response = requests.get(url, headers=headers)
ipo_dfs = pd.read_html(response.text)

In [43]:
#quckily take a look at current dataset, and find I need to convert some into correct data type
ipos_filings = ipo_dfs[0]
ipos_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     325 non-null    object
 1   Symbol          325 non-null    object
 2   Company Name    325 non-null    object
 3   Price Range     325 non-null    object
 4   Shares Offered  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [44]:
# 1. convert the filing date into correct date type
ipos_filings['Filing Date'] = pd.to_datetime(ipos_filings['Filing Date'])
ipos_filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000
1,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
2,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000
3,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-
4,2024-04-22,DRJT,Derun Group Inc,$5.00,-


In [45]:
# 2. deal with shares offers, which should be int data type

## identify if there's any "-" in shared offers
ipos_filings_missing = ipos_filings[ipos_filings['Shares Offered'].astype(str).str.find('-')>=0]
ipos_filings_missing.head()
## replace '-' with NaN
ipos_filings['Shares Offered'] = pd.to_numeric(ipos_filings['Shares Offered'].astype(str).str.replace('-','NaN'), errors='coerce')
ipos_filings['Shares Offered'] = pd.to_numeric(ipos_filings['Shares Offered'])
ipos_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     325 non-null    datetime64[ns]
 1   Symbol          325 non-null    object        
 2   Company Name    325 non-null    object        
 3   Price Range     325 non-null    object        
 4   Shares Offered  253 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 12.8+ KB


In [46]:
# 3. get the avg_price

## def the function

import re

def extract_price(input_string):
    low_range = re.search(r'\$(\d+(?:\.\d+)?)',input_string)
    high_range = re.search(r'-\s*\$\s*(\d+(?:\.\d+)?)', input_string)

    low_number = float(low_range.group(1)) if low_range else None
    high_number = float(high_range.group(1)) if high_range else None

    if low_number is not None and high_number is not None:
       return (low_number + high_number) / 2
    else:
       return low_number or high_number

#Examples
Examples = ["$3.50 - $4.50", "$6.5", "-"]
for example in Examples:
    res = extract_price(example)
    print(f"{example} ==> {res}")

$3.50 - $4.50 ==> 4.0
$6.5 ==> 6.5
- ==> None


In [47]:
## apply the function to price range
ipos_filings['avg_price'] = ipos_filings['Price Range'].apply(lambda x: extract_price(x))
ipos_filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price
0,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.0
1,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.0
2,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.0
3,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN
4,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN,5.0


In [48]:
# 4. get the new column shares_offered_value
ipos_filings['Shares_offered_value'] = (ipos_filings['avg_price']*ipos_filings['Shares Offered']).round().astype('Int64', errors='ignore')
ipos_filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price,Shares_offered_value
0,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.0,11250000
1,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.0,60000000
2,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.0,50000000
3,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN,<NA>
4,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN,5.0,<NA>


In [49]:
# 5. select the target timeframe
ipos_filings = ipos_filings[(ipos_filings['Filing Date']>='2023-01-01') & (ipos_filings['Filing Date']<='2023-12-31')]
ipos_filings_friday = ipos_filings[ipos_filings['Filing Date'].dt.dayofweek==4]
ipos_filings_friday.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 49 to 165
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           32 non-null     datetime64[ns]
 1   Symbol                32 non-null     object        
 2   Company Name          32 non-null     object        
 3   Price Range           32 non-null     object        
 4   Shares Offered        25 non-null     float64       
 5   avg_price             25 non-null     float64       
 6   Shares_offered_value  25 non-null     Int64         
dtypes: Int64(1), datetime64[ns](1), float64(2), object(3)
memory usage: 2.0+ KB


In [50]:
# finally, get the sum value of all non-null shared_offered_value
ipos_filings_friday.Shares_offered_value.sum()

285700000

#Question 2

In [51]:
## 1. get the source data

# input dataset
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

# get 2023 data
url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)
ipo_dfs = pd.read_html(response.text)
ipo_dfs_2023 = ipo_dfs[0]
ipo_dfs_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [52]:
# get 2024 data
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)
ipo_dfs = pd.read_html(response.text)

ipo_dfs_2024 = ipo_dfs[0]
ipo_dfs_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      64 non-null     object
 1   Symbol        64 non-null     object
 2   Company Name  64 non-null     object
 3   IPO Price     64 non-null     object
 4   Current       64 non-null     object
 5   Return        64 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [53]:
ipo_dfs_2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$29.00,20.83%
1,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.17,-20.75%
2,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$32.75,2.34%
3,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$48.16,72.00%
4,"Apr 25, 2024",MRX,Marex Group plc,$19.00,$18.93,-0.37%


In [54]:
## 2. data cleaning and transformation

# join the two dataset and convert the data type into correct one
stacked_ipo_dfs = pd.concat([ipo_dfs_2023,ipo_dfs_2024],ignore_index=True)
stacked_ipo_dfs['IPO Date'] = pd.to_datetime(stacked_ipo_dfs['IPO Date'])
#select the target timeframe
stacked_ipo_dfs = stacked_ipo_dfs[stacked_ipo_dfs['IPO Date']< '2024-03-01']

In [55]:
stacked_ipo_dfs.tail()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
213,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$16.47,0.12%
214,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$2.32,-61.33%
215,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$3.20,-20.00%
216,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$30.21,43.86%
217,2024-01-09,ROMA,Roma Green Finance Limited,$4.00,$0.72,-82.04%


In [56]:
stacked_ipo_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      185 non-null    datetime64[ns]
 1   Symbol        185 non-null    object        
 2   Company Name  185 non-null    object        
 3   IPO Price     185 non-null    object        
 4   Current       185 non-null    object        
 5   Return        185 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 10.1+ KB


In [57]:
#remove symbol = 'RYZB' as it's not in yahoo finance anymore
stacked_ipo_dfs= stacked_ipo_dfs[stacked_ipo_dfs['Symbol'].astype(str).str.find('RYZB')<0]
# renamed one ticker
stacked_ipo_dfs['Symbol'] = stacked_ipo_dfs['Symbol'].astype(str).str.replace('IBAC','IBACU')
stacked_ipo_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      184 non-null    datetime64[ns]
 1   Symbol        184 non-null    object        
 2   Company Name  184 non-null    object        
 3   IPO Price     184 non-null    object        
 4   Current       184 non-null    object        
 5   Return        184 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 10.1+ KB


<ipython-input-57-03312babe555>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stacked_ipo_dfs['Symbol'] = stacked_ipo_dfs['Symbol'].astype(str).str.replace('IBAC','IBACU')


In [58]:
#get the stickers
tickers = stacked_ipo_dfs['Symbol']

In [59]:
## 3. download the the daily price & transformation

import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(tickers):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

   # generate features for historical prices
  historyPrices['Ticker'] = ticker
  historyPrices.index = pd.to_datetime(historyPrices.index)  # Convert index to DateTimeIndex
  historyPrices['Date'] = historyPrices.index.date

   # historical returns
  for i in range(1,31):
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)

  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

COLUMNS = [k for k in stocks_df.keys() if k.find('growth')>=0 or k=='Adj Close' or k == 'Date' or k == 'Ticker']
stocks_df = stocks_df[COLUMNS]

0 IROH


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 LGCB



[*********************100%%**********************]  1 of 1 completed

2 ZKH


...

179 PSBD
180 CCTG


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


181 SYNX
182 SDHC


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

183 ROMA


In [60]:
# count of observations by stock
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37103 entries, 0 to 37102
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Adj Close   37103 non-null  float64
 1   Ticker      37103 non-null  object 
 2   Date        37103 non-null  object 
 3   growth_1d   36920 non-null  float64
 4   growth_2d   36737 non-null  float64
 5   growth_3d   36554 non-null  float64
 6   growth_4d   36371 non-null  float64
 7   growth_5d   36188 non-null  float64
 8   growth_6d   36005 non-null  float64
 9   growth_7d   35822 non-null  float64
 10  growth_8d   35639 non-null  float64
 11  growth_9d   35456 non-null  float64
 12  growth_10d  35273 non-null  float64
 13  growth_11d  35090 non-null  float64
 14  growth_12d  34907 non-null  float64
 15  growth_13d  34724 non-null  float64
 16  growth_14d  34541 non-null  float64
 17  growth_15d  34358 non-null  float64
 18  growth_16d  34175 non-null  float64
 19  growth_17d  33993 non-nul

In [61]:
# how many days records each stock has
stocks_df.Ticker.value_counts()

Ticker
ATS     3611
ENLT     336
MGOL     328
SKWD     328
CVKD     324
        ... 
VHAI      49
SMXT      48
JVSA      38
LEGT      26
DYCQ      16
Name: count, Length: 183, dtype: int64

In [62]:
#timeframe for each stock
stocks_df.groupby(['Ticker']).Date.agg(['min','max'])

,min,max
Ticker,,
AACT,2023-06-13,2024-05-03
AESI,2023-03-09,2024-05-03
AFJK,2024-01-23,2024-05-02
AHR,2024-02-07,2024-05-03
AITR,2024-01-02,2024-05-03
...,...,...
WRNT,2023-07-25,2024-05-03
YGFGF,2023-03-28,2024-05-02
YIBO,2024-01-25,2024-05-03


In [63]:
#convert into right data type
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])
stocks_df['Ticker'] = stocks_df['Ticker'].astype(str)

In [64]:
stocks_df.head()

,Adj Close,Ticker,Date,growth_1d,growth_2d,growth_3d,growth_4d,growth_5d,growth_6d,growth_7d,...,growth_21d,growth_22d,growth_23d,growth_24d,growth_25d,growth_26d,growth_27d,growth_28d,growth_29d,growth_30d
0,10.010,IROH,2024-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10.020,IROH,2024-02-20,1.000999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.015,IROH,2024-02-21,0.999501,1.000500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10.020,IROH,2024-02-22,1.000499,1.000000,1.000999,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10.010,IROH,2024-02-23,0.999002,0.999501,0.999002,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37103 entries, 0 to 37102
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Adj Close   37103 non-null  float64       
 1   Ticker      37103 non-null  object        
 2   Date        37103 non-null  datetime64[ns]
 3   growth_1d   36920 non-null  float64       
 4   growth_2d   36737 non-null  float64       
 5   growth_3d   36554 non-null  float64       
 6   growth_4d   36371 non-null  float64       
 7   growth_5d   36188 non-null  float64       
 8   growth_6d   36005 non-null  float64       
 9   growth_7d   35822 non-null  float64       
 10  growth_8d   35639 non-null  float64       
 11  growth_9d   35456 non-null  float64       
 12  growth_10d  35273 non-null  float64       
 13  growth_11d  35090 non-null  float64       
 14  growth_12d  34907 non-null  float64       
 15  growth_13d  34724 non-null  float64       
 16  growth_14d  34541 non-

In [87]:
stocks_df.describe()
# x=30? growth is 1.068557

,Adj Close,Date,growth_1d,growth_2d,growth_3d,growth_4d,growth_5d,growth_6d,growth_7d,growth_8d,...,growth_21d,growth_22d,growth_23d,growth_24d,growth_25d,growth_26d,growth_27d,growth_28d,growth_29d,growth_30d
count,37103.000000,37103,36920.000000,36737.000000,36554.000000,36371.000000,36188.000000,36005.000000,35822.000000,35639.000000,...,33265.000000,33083.000000,32901.000000,32719.000000,32537.000000,32355.000000,32174.000000,31993.000000,31812.000000,31631.000000
mean,12.247741,2023-03-29 10:44:13.305662464,0.999563,0.999364,0.999270,0.999095,0.999093,0.999155,0.998953,0.998820,...,0.994459,0.994098,0.993648,0.993281,0.992987,0.992920,0.992799,0.992581,0.992463,0.992484
min,0.002000,2009-12-29 00:00:00,0.028674,0.003584,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,...,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729
25%,1.830000,2023-07-28 00:00:00,0.977379,0.966135,0.957265,0.949153,0.941438,0.933765,0.926783,0.919974,...,0.839744,0.834686,0.830056,0.823302,0.819350,0.813471,0.810231,0.806180,0.802061,0.798330
50%,7.810000,2023-11-22 00:00:00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.996310,0.995904,0.995604,0.995131,0.995227,0.994290,0.994077,0.993952,0.993034,0.993031
75%,12.030000,2024-02-21 00:00:00,1.014925,1.020270,1.025244,1.028307,1.031844,1.034005,1.035860,1.038976,...,1.058491,1.059669,1.060222,1.061011,1.061921,1.061607,1.063811,1.065483,1.067225,1.068557
max,4318.000000,2024-05-03 00:00:00,6.826923,6.971311,7.625593,8.650538,8.952632,10.309092,10.183544,10.380646,...,12.634409,11.491442,12.486842,12.284789,12.661000,15.025219,14.782093,19.655310,22.918785,27.198465
std,40.725151,NaN,0.095948,0.135521,0.166629,0.191294,0.215651,0.240397,0.258317,0.276748,...,0.433825,0.444211,0.454068,0.463155,0.473509,0.487012,0.500588,0.517693,0.537497,0.560095


In [67]:
## additional: if I want to get the growth in 30 days except the 'LEGT', 'DYCQ' not arrive 30 days?? since lack of data

# def get_target_date_rows():
#     target_date_rows = pd.DataFrame()  # Initialize an empty DataFrame to store target date rows

#     # Iterate over unique tickers
#     for ticker in stocks_df['Ticker'].unique():
#         # Filter DataFrame for the specified ticker
#         ticker_df = stocks_df[stocks_df['Ticker'] == ticker]
#         # Find the minimum date for the specified ticker
#         min_date = min(ticker_df['Date'])
#         # Calculate the number of occurrences of the ticker in the DataFrame
#         ticker_count = ticker_df['Ticker'].value_counts()[ticker] if ticker in ['LEGT', 'DYCQ'] else 30
#         # Calculate the target date based on the minimum date and the count of occurrences
#         target_date = min_date + pd.DateOffset(days=int(ticker_count))
#         # Filter DataFrame for rows with the target date and append to target_date_rows
#         target_date_row = ticker_df[ticker_df['Date'] == target_date]
#         target_date_rows = pd.concat([target_date_rows, target_date_row], ignore_index=True)

#     return target_date_rows

# # Call the function to get the rows for the target date for each ticker
# target_date_rows = get_target_date_rows()

# # Print the rows for the target date for each ticker
# print("Rows for the target date for each ticker:")
# print(target_date_rows)

## Question 3

In [68]:
## 1. downloard the OHLCV data

# LARGET Companies
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

largest_stocks = US_STOCKS + EU_STOCKS + INDIA_STOCKS
print(largest_stocks)

['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO', 'V', 'JPM', 'NVO', 'MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE', 'IDEXY', 'CDI.PA', 'RELIANCE.NS', 'TCS.NS', 'HDB', 'BHARTIARTL.NS', 'IBN', 'SBIN.NS', 'LICI.NS', 'INFY', 'ITC.NS', 'HINDUNILVR.NS', 'LT.NS']


In [69]:
# download and get required columns

import time

largest_stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(largest_stocks):
  print(i,ticker)

  # Work with stock prices
  largest_stocks_historcial_prices = yf.download(tickers = ticker,
                     start= "2013-12-20", end= "2023-12-31",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  largest_stocks_historcial_prices['Ticker'] = ticker
  largest_stocks_historcial_prices.index = pd.to_datetime(largest_stocks_historcial_prices.index)  # Convert index to DateTimeIndex
  largest_stocks_historcial_prices['Date'] = largest_stocks_historcial_prices.index.date

  # get the growth_7d
  largest_stocks_historcial_prices['Growth_7d'] = largest_stocks_historcial_prices['Adj Close'] / largest_stocks_historcial_prices['Adj Close'].shift(7)

  if largest_stocks_df.empty:
    largest_stocks_df = largest_stocks_historcial_prices
  else:
    largest_stocks_df = pd.concat([largest_stocks_df, largest_stocks_historcial_prices], ignore_index=True)

# get the necessary columns
COLUMNS = [k for k in largest_stocks_df.keys() if k !='Open' and k!='High' and k!='Low' and k!='Close']
largest_stocks_df = largest_stocks_df[COLUMNS]
print(largest_stocks_df.Ticker.unique())


0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


2 GOOG
3 NVDA


...

31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed

['MSFT' 'AAPL' 'GOOG' 'NVDA' 'AMZN' 'META' 'BRK-B' 'LLY' 'AVGO' 'V' 'JPM'
 'NVO' 'MC.PA' 'ASML' 'RMS.PA' 'OR.PA' 'SAP' 'ACN' 'TTE' 'SIE.DE' 'IDEXY'
 'CDI.PA' 'RELIANCE.NS' 'TCS.NS' 'HDB' 'BHARTIARTL.NS' 'IBN' 'SBIN.NS'
 'LICI.NS' 'INFY' 'ITC.NS' 'HINDUNILVR.NS' 'LT.NS']


In [70]:
# transform into right datatype
largest_stocks_df['Date']=pd.to_datetime(largest_stocks_df['Date'])
largest_stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80979 entries, 0 to 80978
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Adj Close  80979 non-null  float64       
 1   Volume     80979 non-null  int64         
 2   Ticker     80979 non-null  object        
 3   Date       80979 non-null  datetime64[ns]
 4   Growth_7d  80748 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 3.1+ MB


In [71]:
# get the first avg growth_7d
print(largest_stocks_df[largest_stocks_df['Date'] == '2014-01-01']['Growth_7d'].mean())

1.0117972682083245


In [72]:
## 2. prepare the large_stocks

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

large_stocks = NEW_EU + NEW_US + NEW_INDIA

In [73]:
# download and get required columns

import time

large_stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(large_stocks):
  print(i,ticker)

  # Work with stock prices
  large_stocks_historcial_prices = yf.download(tickers = ticker,
                     start= "2013-12-20", end= "2024-04-30",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  large_stocks_historcial_prices['Ticker'] = ticker
  large_stocks_historcial_prices.index = pd.to_datetime(large_stocks_historcial_prices.index)  # Convert index to DateTimeIndex
  large_stocks_historcial_prices['Date'] = large_stocks_historcial_prices.index.date

  # get the growth_7d
  large_stocks_historcial_prices['Growth_7d'] = large_stocks_historcial_prices['Adj Close'] / large_stocks_historcial_prices['Adj Close'].shift(7)

  if large_stocks_df.empty:
    large_stocks_df = large_stocks_historcial_prices
  else:
    large_stocks_df = pd.concat([large_stocks_df, large_stocks_historcial_prices], ignore_index=True)

# get the necessary columns
COLUMNS = [k for k in large_stocks_df.keys() if k !='Open' and k!='High' and k!='Low' and k!='Close']
large_stocks_df = large_stocks_df[COLUMNS]
print(large_stocks_df.Ticker.unique())

0 PRX.AS


[*********************100%%**********************]  1 of 1 completed

1 CDI.PA



[*********************100%%**********************]  1 of 1 completed


2 AIR.PA


...

32 KOTAKBANK.NS


[*********************100%%**********************]  1 of 1 completed


33 TITAN.NS


[*********************100%%**********************]  1 of 1 completed

['PRX.AS' 'CDI.PA' 'AIR.PA' 'SU.PA' 'ETN' 'SNY' 'BUD' 'DTE.DE' 'ALV.DE'
 'MDT' 'AI.PA' 'EL.PA' 'TSLA' 'WMT' 'XOM' 'UNH' 'MA' 'PG' 'JNJ' 'MRK' 'HD'
 'COST' 'ORCL' 'BAJFINANCE.NS' 'MARUTI.NS' 'HCLTECH.NS' 'TATAMOTORS.NS'
 'SUNPHARMA.NS' 'ONGC.NS' 'ADANIENT.NS' 'NTPC.NS' 'KOTAKBANK.NS'
 'TITAN.NS']


In [74]:
# transform into right datatype
large_stocks_df['Date']=pd.to_datetime(large_stocks_df['Date'])
large_stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86828 entries, 0 to 86827
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Adj Close  86828 non-null  float64       
 1   Volume     86828 non-null  int64         
 2   Ticker     86828 non-null  object        
 3   Date       86828 non-null  datetime64[ns]
 4   Growth_7d  86590 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 3.3+ MB


In [75]:
# get the first avg growth_7d
print(large_stocks_df[large_stocks_df['Date'] == '2014-01-01']['Growth_7d'].mean())

1.0116841967195396


In [76]:
## 3. group by date and merge two tables

#for large_stocks
large_stocks_daily_avg_growth_7d = pd.DataFrame(large_stocks_df.groupby(['Date']).Growth_7d.agg('mean')).reset_index()
large_stocks_daily_avg_growth_7d['Date']=pd.to_datetime(large_stocks_daily_avg_growth_7d['Date'])
large_stocks_daily_avg_growth_7d=large_stocks_daily_avg_growth_7d[large_stocks_daily_avg_growth_7d['Date']>="2014-01-01"]
large_stocks_daily_avg_growth_7d.rename(columns={'Growth_7d': 'large_stocks_growth_7d'}, inplace=True)
large_stocks_daily_avg_growth_7d.info()

#for largest_stocks
largest_stocks_daily_avg_growth_7d = pd.DataFrame(largest_stocks_df.groupby(['Date']).Growth_7d.agg('mean')).reset_index()
largest_stocks_daily_avg_growth_7d['Date']=pd.to_datetime(largest_stocks_daily_avg_growth_7d['Date'])
largest_stocks_daily_avg_growth_7d=largest_stocks_daily_avg_growth_7d[largest_stocks_daily_avg_growth_7d['Date']>="2014-01-01"]
largest_stocks_daily_avg_growth_7d.rename(columns={'Growth_7d': 'largest_stocks_growth_7d'}, inplace=True)
largest_stocks_daily_avg_growth_7d.info()

#merge the two tables
all_stocks_growth_7d = pd.merge(largest_stocks_daily_avg_growth_7d, large_stocks_daily_avg_growth_7d, how='left', on='Date')

<class 'pandas.core.frame.DataFrame'>
Index: 2680 entries, 7 to 2686
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    2680 non-null   datetime64[ns]
 1   large_stocks_growth_7d  2680 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 62.8 KB
<class 'pandas.core.frame.DataFrame'>
Index: 2595 entries, 7 to 2601
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      2595 non-null   datetime64[ns]
 1   largest_stocks_growth_7d  2595 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 60.8 KB


In [77]:
all_stocks_growth_7d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2595 entries, 0 to 2594
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      2595 non-null   datetime64[ns]
 1   largest_stocks_growth_7d  2595 non-null   float64       
 2   large_stocks_growth_7d    2595 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 60.9 KB


In [78]:
all_stocks_growth_7d.head()

,Date,largest_stocks_growth_7d,large_stocks_growth_7d
0,2014-01-01,1.011797,1.011684
1,2014-01-02,1.002964,0.997706
2,2014-01-03,0.997668,0.999534
3,2014-01-06,0.993423,0.995876
4,2014-01-07,0.992509,0.991592


In [79]:
## 4. find how many days large_stocks growth > largest_stocks_growth --- 47%
round(((all_stocks_growth_7d['large_stocks_growth_7d']-all_stocks_growth_7d['largest_stocks_growth_7d'])>0).sum() / 2595 * 100, 0)

47.0

## Question 4

In [80]:
## 1. download AND prepare the dataframe

from google.colab import drive
drive.mount('/content/drive')
parquet_path = '/content/stocks_df_combined_trunc_2014_2023.parquet.brotli'

all_stocks_df = pd.read_parquet(parquet_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
# https://stackoverflow.com/questions/49648391/how-to-install-ta-lib-in-google-colab
# Update (apr 2023): Colab is now Python 3.10
# there are some other (older) ways to install talib

url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib as ta

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4127    0  4127    0     0  14189      0 --:--:-- --:--:-- --:--:-- 14231
100  517k  100  517k    0     0   705k      0 --:--:-- --:--:-- --:--:-- 2651k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4087    0  4087    0     0  19815      0 --:--:-- --:--:-- --:--:-- 19839
100  392k  100  392k    0     0   753k      0 --:--:-- --:--:-- --:--:--  753k


In [82]:
all_stocks_df=all_stocks_df[(all_stocks_df['Date'] <='2023-12-31') & (all_stocks_df['Date'] >='2014-01-01')]

In [83]:
all_stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80762 entries, 7011 to 5342
Columns: 202 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(128), int32(64), int64(5), object(2)
memory usage: 105.4+ MB


In [84]:
# 2. Commodity Channel Index Python Code

  # df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3
  # df['sma'] = df['TP'].rolling(ndays).mean()
  # df['mad'] = df['TP'].rolling(ndays).apply(lambda x: pd.Series(x).mad())
  # df['CCI'] = (df['TP'] - df['sma']) / (0.015 * df['mad'])

# Load the necessary libraries
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt


# Downloading sample data for multiple stocks
tickers = all_stocks_df['Ticker'].unique()  # Example list of stock tickers
start_date = "2014-01-01"
end_date = "2023-12-31"

# Function to calculate CCI and profits for each 7 day trade for each stock
def all_stock_CCI(df, ndays):
    df['CCI'] = ta.CCI(df['High'], df['Low'], df['Close'], timeperiod=ndays)
    df['Profits'] = (1000/df['Adj Close_x']) * (df['Adj Close_x'].shift(-5)) - 1000
    return df

# Function to filter CCI values greater than 200 on Fridays
def filter_CCI(df):
    df['DayOfWeek'] = df.Date.dt.dayofweek  # Add column for day of week (0=Monday, 6=Sunday)
    return df[(df['CCI'] > 200) & (df['DayOfWeek'] == 4)]  # Select CCI>200 on Fridays (DayOfWeek=4)

# Create an empty DataFrame to store results
filtered_results = pd.DataFrame()

# Iterate over each stock ticker
for ticker in tickers:
    # Download historical data for the current stock
    stock_data = all_stocks_df

    # Calculate CCI for the current stock
    stock_data = all_stock_CCI(stock_data, ndays=14)  # Adjust ndays as needed

    # Filter CCI values greater than 200 on Fridays for the current stock
    filtered_data = filter_CCI(stock_data)

    # Append filtered results to the DataFrame
    final_results = pd.concat([filtered_results, filtered_data],ignore_index=True)

COLUMNS = [k for k in final_results.keys() if k == 'Ticker' or k=='Adj Close_x' or k=='Date' or k=='CCI' or k=='Profits']
final_results = final_results[COLUMNS]
final_results.head(10)

,Adj Close_x,Ticker,Date,CCI,Profits
0,34.912762,MSFT,2014-05-30,203.518798,13.190094
1,39.395618,MSFT,2014-09-05,257.013330,17.207263
2,41.630741,MSFT,2015-04-24,340.009871,16.503151
3,40.151123,MSFT,2015-10-02,206.125828,33.793947
4,46.583046,MSFT,2015-10-23,431.107024,-4.350092
5,48.866241,MSFT,2016-01-29,250.884243,-89.490064
6,54.007221,MSFT,2016-10-21,349.230464,3.519788
7,59.949631,MSFT,2017-01-27,257.258566,-31.924599
8,60.881535,MSFT,2017-04-21,212.247487,31.024462
9,77.696846,MSFT,2017-10-27,330.634327,3.937497


In [85]:
final_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Adj Close_x  461 non-null    float64       
 1   Ticker       461 non-null    object        
 2   Date         461 non-null    datetime64[ns]
 3   CCI          461 non-null    float64       
 4   Profits      461 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 18.1+ KB


In [86]:
final_results['Profits'].sum()

108.35117767507495